** Load Libraries **

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.sandbox.tools import pca
import numpy as np

** Load data **

In [ ]:
predictors = pd.read_table('Predictors.txt',sep=';')
response = pd.read_table('Response.txt',sep=';')
# Correlation matrix with Y
frame = pd.merge(response, predictors, left_index=True, right_index=True, how='inner')
frame.rename(columns={'x': 'Y'}, inplace=True)
frame_1 = frame[:100]
frame_corr = frame_1.corr()
frame_corr = frame_corr.sort(['Y'], ascending=[0])
frame_corr = frame_corr[abs(frame_corr.Y) > 0.10] #63 covariates
imp_covariates = frame_corr['Y'].index.tolist()
# Correlation matrix of covariates
predictors_corr = predictors.corr()
frame_1 = frame_1[imp_covariates]

** Train and Test data Split **

In [94]:
#x_train, x_test, y_train, y_test = train_test_split(frame.ix[:, frame_1.columns != 'Y'], frame_1['Y'], test_size=0.40, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(frame_1.ix[:100, frame_1.columns != 'Y'], frame_1.Y[:100], test_size=0.30, random_state=42)

** Fit the LASSO **

In [110]:
lasso = linear_model.Lasso(alpha=0.1)
lasso.fit(x_train,y_train)
lasso.coef_
lasso.predict(frame.ix[101:200, frame_1.columns != 'Y'])

array([ 11.22532208,  10.79997026,   9.85850095,  11.95704227,
         7.86793096,  10.07900775,   9.99800419,   8.33798311,
         5.82159642,   8.98121409,  12.64806921,  11.18212891,
         8.7726332 ,   7.44547415,   9.95479163,   8.00978811,
        11.14346   ,  11.00328059,   6.94770625,  10.10430002,
         8.0536641 ,   6.16288331,   9.92205946,  10.10302751,
         6.67043452,  12.19156748,   9.06254609,   4.67826079,
        11.98409661,   8.3472421 ,  10.02895756,   8.68944833,
         9.05403421,  11.02177455,  13.05491152,  12.99114103,
         6.0006568 ,  11.43088534,  10.68401521,   8.40073646,
         6.15357018,  10.32779381,  11.62351743,   8.28457253,
        12.71583697,  10.02699414,  10.48188626,  11.36745582,
        12.17846781,   9.75851555,  11.85794966,   8.37448947,
         9.09748631,   9.23082987,  11.15051913,  10.84461787,
        10.28886959,  10.34001212,   9.80780646,   9.48021361,
        12.91196058,   8.5555506 ,   9.06596046,  10.95

In [111]:
print("Average Squared Error",np.mean((lasso.predict(x_test) - y_test) ** 2))

('Average Squared Error', 6.3221200598893414)


In [112]:
print('R^2 or Variance score: %.2f' % lasso.score(x_test, y_test))

R^2 or Variance score: 0.19


** Principle Componenet Analysis **

In [ ]:
"""frame_2 = frame[frame.columns.delete([0])]
frame_2 = frame_2[:100] 
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(frame_2)

cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)"""

** Grouping of Brain hot-spots by pointrank method **

In [ ]:
co_ordinates = pd.DataFrame(frame.columns.values)
co_ordinates = co_ordinates.drop(co_ordinates.index[:1])
co_ordinates = co_ordinates.reset_index(drop=True)
x_p = np.array(([x for x in range(1,21)]))
y_p = np.array(([x for x in range(1,21)]))
z_p = np.array(([x for x in range(1,21)]))
co_od = np.vstack(np.meshgrid(x_p,y_p,z_p)).reshape(3,-1).T
co_ordinates['u'] = co_od[:,0]
co_ordinates['v'] = co_od[:,1]
co_ordinates['w'] = co_od[:,2]

reduced_co_ordinates = pd.DataFrame(frame_1.columns.values)
reduced_co_ordinates = reduced_co_ordinates.drop(co_ordinates.index[:1])
reduced_co_ordinates = reduced_co_ordinates.reset_index(drop=True)
reduced_co_ordinates = pd.merge(reduced_co_ordinates, co_ordinates,on = [0], how='left')
reduced_co_ordinates.rename(columns={0: 'co_od'}, inplace=True)

def euclidean_dist(x):
    if np.linalg.norm(np.array((x[0] ,x[1], x[2]))-np.array((x[3] ,x[4], x[5]))) < 5.0:
        return 1 
    else: return 0

def pointRankComputation(reduced_co_ordinates):
    reduced_co_ordinates.loc[:,('pointRank')] = np.nan
    for index,row in reduced_co_ordinates.iterrows():
        temp = pd.DataFrame({"1" : reduced_co_ordinates.loc[index,'u'],"2": reduced_co_ordinates.loc[index,'v'],"3": reduced_co_ordinates.loc[index,'w'],"4": reduced_co_ordinates['u'],"5": reduced_co_ordinates['v'],"6": reduced_co_ordinates['w']})
        temp['dist'] = temp.apply(euclidean_dist,axis=1)
        reduced_co_ordinates.loc[index,'pointRank'] = np.sum(temp['dist'])

pointRankComputation(reduced_co_ordinates)

reduced_co_ordinates = reduced_co_ordinates.sort(['pointRank'], ascending=[0])
reduced_co_ordinates = reduced_co_ordinates.reset_index(drop=True)

a= []
b= []
reduced_co_ordinates['group'] = 0
cntr = 1
for index in range(len(reduced_co_ordinates)):
    if reduced_co_ordinates.loc[index,'group'] == 0:
        print index
        temp = pd.DataFrame({"1" : reduced_co_ordinates.loc[index,'u'],"2": reduced_co_ordinates.loc[index,'v'],"3": reduced_co_ordinates.loc[index,'w'],"4": reduced_co_ordinates['u'],"5": reduced_co_ordinates['v'],"6": reduced_co_ordinates['w']})
        temp['flag'] = temp.apply(euclidean_dist,axis=1)
        a = []
        a = temp[temp.flag == 1].index.tolist()
        for j in range(len(a)):
            try:
                if cntr == 1:
                    reduced_co_ordinates.loc[a[j],'group'] = cntr
                else: 
                    new_list = [x for x in a if x not in b]
                    reduced_co_ordinates.loc[new_list[j],'group'] = cntr
            except IndexError: pass
        b.extend(a)       
        try:       
            b = [item for sublist in b for item in sublist] #flatten       
        except TypeError: pass       
        b = [e for i, e in enumerate(b) if b.index(e) == i] #unique       
        cntr += 1

yo = pd.DataFrame(frame_1.columns.tolist(),columns=['co_od'])
dfdf = pd.merge(yo,reduced_co_ordinates,on=['co_od'],how='left')

** Grouped - LASSO**

In [86]:
# -*- coding: utf-8 -*-
"""
Strong rules for coordinate descent
Author: Fabian Pedregosa <fabian@fseoane.net>
"""

import numpy as np
from scipy import linalg

MAX_ITER = 100


def l1_coordinate_descent(X, y, alpha, warm_start=None, max_iter=MAX_ITER):

    if warm_start is not None:
        beta = warm_start
    else:
        beta = np.zeros(X.shape[1], dtype=np.float)
    alpha = alpha * X.shape[0]

    for _ in range(max_iter):
        for i in range(X.shape[1]):
            bb = beta.copy()
            bb[i] = 0.
            residual = np.dot(X[:, i], y - np.dot(X, bb).T)
            beta[i] = np.sign(residual) * np.fmax(np.abs(residual) - alpha, 0) \
                / np.dot(X[:, i], X[:, i])
    return beta


def shrinkage(X, y, alpha, beta, active_set, max_iter):
    bb = beta.copy()
    for _ in range(max_iter):
        for i in active_set:

            bb[i] = 0
            residual = np.dot(X[:, i], y - np.dot(X, bb).T)
            bb[i] = np.sign(residual) * np.fmax(np.abs(residual) - alpha, 0) \
                / np.dot(X[:, i], X[:, i])
    return bb


def l1_path(X, y, alphas, max_iter=MAX_ITER, verbose=False):
    """
    The strategy is described in "Strong rules for discarding predictors in lasso-type problems"
    alphas must be a decreasing sequence of regularization parameters
    WARNING: does not compute intercept
    """
    beta = np.zeros((len(alphas), X.shape[1]), dtype=np.float)

    alphas_scaled = np.array(alphas) * X.shape[0]

    active_set = np.arange(X.shape[1]).tolist()
    for k, a in enumerate(alphas_scaled):

        if k > 0:
        # .. Strong rules for discarding predictors in lasso-type ..
            tmp = np.dot(X.T, y - np.dot(X, beta[k - 1]))
            tmp = np.abs(tmp)
            strong_active_set = tmp >= 2 * alphas_scaled[k] - alphas_scaled[k - 1]
            strong_active_set = np.where(strong_active_set)[0]
        else:
            strong_active_set = np.arange(X.shape[1])

        if verbose:
	    print 'Strong active set ', strong_active_set
	    print 'Active set ', active_set
        # solve for the current active set
        beta[k] = shrinkage(X, y, a, beta[k], active_set, max_iter)

        # check KKT in the strong active set
        kkt_violations = False
        for i in strong_active_set:
            tmp = np.dot(X[:, i], y - np.dot(X, beta[k]))
            if beta[k, i] != 0 and not np.allclose(tmp, np.sign(beta[k, i]) * alphas_scaled[k]):
                if i not in active_set:
                    active_set.append(i)
                kkt_violations = True
            if beta[k, i] == 0 and abs(tmp) >= np.abs(alphas_scaled[k]):
                if i not in active_set:
                    active_set.append(i)
                kkt_violations = True

        if not kkt_violations:
            # passed KKT for all variables in strong active set, we're done
            active_set = np.where(beta[k] != 0)[0].tolist()
            if verbose:
                print 'no KKT violated on strong active set'

        # .. recompute with new active set ..
        else:
            if verbose:
                print 'KKT violated on strong active set'
            beta[k] = shrinkage(X, y, a, beta[k], active_set, max_iter)

        # .. check KKT on all predictors ..
        kkt_violations = False
        for i in range(X.shape[1]):
            tmp = np.dot(X[:, i], y - np.dot(X, beta[k]))
            if beta[k, i] != 0 and not np.allclose(tmp, np.sign(beta[k, i]) * alphas_scaled[k]):
                if i not in active_set:
                    active_set.append(i)
                kkt_violations = True
            if beta[k, i] == 0 and abs(tmp) >= np.abs(alphas_scaled[k]):
                if i not in active_set:
                    active_set.append(i)
                kkt_violations = True

        if not kkt_violations:
            # passed KKT for all variables, we're done
            active_set = np.where(beta[k] != 0)[0].tolist()
            print 'no KKT violated on full active set'
        else:
            if verbose:
                print 'KKT violated on full active set'
            beta[k] = shrinkage(X, y, a, beta[k], active_set, max_iter)


    return beta

def check_kkt_lasso(xr, coef, penalty, tol=1e-3):
    """
    Check KKT conditions for Lasso
    xr : X'(y - X coef)
    """
    nonzero = (coef != 0)
    return np.all(np.abs(xr[nonzero] - np.sign(coef[nonzero]) * penalty) < tol) \
        and np.all(np.abs(xr[~nonzero] / penalty) <= 1)


In [87]:
if __name__ == '__main__':
    X = np.array(x_train)
    y = np.array(y_train)
    alpha = .1
    #groups = np.r_[[0, 0], [0, 0, 1, 8, 9, 5, 6, 7]]
    groups = groups = np.r_[[1,2,3,4,5] * 1600]
    coefs = group_lasso(X, y, alpha, groups, verbose=True)
    #print 'KKT conditions verified:', check_kkt(X, y, coefs, alpha, groups)

ValueError: matrices are not aligned

In [ ]:
arr1 = lasso.intercept_+np.dot(x_test,coefs)
print("Average Squared Error",np.mean((arr1 - y_test) ** 2))

In [ ]:
len(groups)

In [81]:
np.array(x_train).shape

(70, 8000)

In [83]:
y_train.shape

(70,)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(frame.ix[:, frame_1.columns != 'Y'], frame_1['Y'], test_size=0.40, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(frame.ix[:100, frame.columns != 'Y'], frame.Y[:100], test_size=0.30, random_state=42)

In [77]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

In [109]:
frame.ix[101:200, frame_1.columns != 'Y']

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X2431,X2432,X2433,X2434,X2435,X2436,X2437,X2438,X2439,X2440
101,1.04,1.10,0.50,-0.32,-0.69,0.19,0.08,0.97,0.74,1.32,...,0.14,-0.25,-0.41,-0.21,-1.50,-1.36,0.70,-0.28,-0.26,0.35
102,-1.09,0.06,1.32,0.93,0.80,1.75,1.05,-1.07,-0.85,-1.21,...,-0.05,-0.57,-2.20,-0.57,-0.28,0.46,-1.65,-2.50,-0.86,-0.58
103,1.46,2.28,0.80,0.68,1.28,1.08,-0.62,-0.83,1.13,-0.29,...,0.36,0.65,1.22,0.80,-0.48,1.35,-0.41,1.69,2.98,2.63
104,2.01,2.43,1.21,-0.12,0.00,-0.24,-1.30,-0.92,-0.48,0.29,...,0.75,0.48,0.74,-0.41,-0.02,-0.19,-1.18,-0.84,-0.45,0.90
105,-0.35,0.06,0.34,-0.99,-1.03,-0.80,-0.02,-0.10,0.26,1.66,...,-0.14,0.25,-1.38,-1.71,-2.19,-1.93,-1.48,-0.13,-0.69,0.21
106,-0.66,0.02,1.56,1.27,2.25,1.17,-0.44,-0.77,0.21,0.49,...,0.48,-1.07,-0.88,0.78,0.38,1.23,1.35,0.30,1.23,-0.19
107,1.18,-1.13,-0.60,-0.63,0.09,-0.73,-0.33,-0.21,-0.10,0.25,...,0.70,0.34,-0.57,0.46,0.22,0.24,-1.64,-1.51,0.86,-0.61
108,-0.79,-0.09,-0.39,-1.00,0.10,0.30,0.31,0.13,1.31,-0.93,...,0.93,0.89,2.17,1.49,3.38,2.15,1.06,-0.05,0.88,-0.68
109,-0.22,-0.58,-1.17,-2.18,-1.01,-0.15,0.02,-0.47,-0.27,-0.21,...,1.91,1.68,1.38,0.22,1.06,1.38,-0.21,-0.88,-0.18,1.23
110,-1.42,-1.82,-1.45,-0.40,-0.29,-0.34,0.63,1.04,0.40,-0.07,...,0.00,0.42,1.22,1.27,-0.70,-2.12,-0.52,-1.58,-1.58,-1.09


In [124]:
x = pd.DataFrame(lasso.predict(frame.ix[101:200, frame_1.columns != 'Y']),columns=['predicted_ma'])

In [129]:
x = x.sort(['predicted_ma'], ascending=[0])

In [149]:
x[:10]

,predicted_ma
81,14.938253
34,13.054912
35,12.991141
60,12.911961
97,12.768238
44,12.715837
79,12.686992
10,12.648069
65,12.251262
25,12.191567


In [146]:
top10

Int64Index([81, 34, 35, 60, 97, 44, 79, 10, 65, 25], dtype='int64')